In [20]:
from modules.dataloader import load_data

boards_train, labels_train, boards_test, labels_test = load_data("../hex_data.csv", 0.9)
dimensions = 5

Number of boards: 3000
First board: OXXEOOOXEXXOXXEOOXEOEXXOO
First label: 0
Training samples:  2700
Test samples:  300
First training board:  OXXEOOOXEXXOXXEOOXEOEXXOO
First training label:  0
First test board:  XOXOOXOOXXXOEXXOOOXXOOOEX
First test label:  1


In [21]:
# setting up training graphs

from GraphTsetlinMachine.graphs import Graphs

#symbols for nodes
symbols = ["X", "O", "E"]

#initialize the graphs object
graphs_train = Graphs(
    number_of_graphs=len(boards_train),
    symbols=symbols
)

#e.g. 49 for 7x7 board
number_of_nodes = dimensions * dimensions

#set the number of nodes for each graph
for graph_id in range(len(boards_train)):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_nodes)


#function to get neighbors of a node (cell)
def get_neighbors(row, column, dimensions):
    neighbors = []
    potential_neighbors = [
        (row - 1, column),  #north-west
        (row + 1, column),  #south-east
        (row, column - 1),  #west
        (row, column + 1),  #east
        (row - 1, column + 1),  #north-east
        (row + 1, column - 1),  #south-west
    ]

    for neighbor_row, neighbor_column in potential_neighbors:
        if 0 <= neighbor_row < dimensions and 0 <= neighbor_column < dimensions:
            neighbors.append((neighbor_row, neighbor_column))

    return neighbors


graphs_train.prepare_node_configuration()

#add the nodes to each graph
for graph_id, board in enumerate(boards_train):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        graphs_train.add_graph_node(graph_id, node_id, len(neighbors))

graphs_train.prepare_edge_configuration()

#add the edges
for graph_id, board in enumerate(boards_train):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        for (neighbor_row, neighbor_column) in neighbors:
            neighbor_id = neighbor_row * dimensions + neighbor_column

            graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id, "adjacent_cell")

#add the node properties
for graph_id, board in enumerate(boards_train):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions

        #X means cell is occupied by player 0, O is player 1, and E is empty
        cell_value = board[node_id]
        if cell_value == 'X':
            graphs_train.add_graph_node_property(graph_id, node_id, "X")
        elif cell_value == 'O':
            graphs_train.add_graph_node_property(graph_id, node_id, "O")
        elif cell_value == 'E':
            graphs_train.add_graph_node_property(graph_id, node_id, "E")

graphs_train.encode()

In [22]:
#setting up the test graphs
#its the same as the training graphs but we need to set them up separately

graphs_test = Graphs(
    number_of_graphs=len(boards_test),
    init_with=graphs_train
)

for graph_id in range(len(boards_test)):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_nodes)

graphs_test.prepare_node_configuration()

#add nodes
for graph_id, board in enumerate(boards_test):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        graphs_test.add_graph_node(graph_id, node_id, len(neighbors))

graphs_test.prepare_edge_configuration()

#add node edges
for graph_id, board in enumerate(boards_test):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        neighbors = get_neighbors(row, column, dimensions)

        for (neighbor_row, neighbor_column) in neighbors:
            neighbor_id = neighbor_row * dimensions + neighbor_column

            graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id, "adjacent_cell")

#add node properties
for graph_id, board in enumerate(boards_test):
    for node_id in range(number_of_nodes):
        row = node_id // dimensions
        column = node_id % dimensions
        cell_value = board[node_id]

        if cell_value == 'X':
            graphs_test.add_graph_node_property(graph_id, node_id, "X")
        elif cell_value == 'O':
            graphs_test.add_graph_node_property(graph_id, node_id, "O")
        elif cell_value == 'E':
            graphs_test.add_graph_node_property(graph_id, node_id, "E")

graphs_test.encode()

In [23]:
#training logic
from modules.trainer import train

train(
    graphs_train=graphs_train,
    labels_train=labels_train,
    graphs_test=graphs_test,
    labels_test=labels_test,
    dimensions=dimensions,
    number_of_clauses=500,
    T=5000,
    s=1.0,
    depth=2,
    accuracy_threshhold=1,
    epochs=50
)

Initialization of sparse structure.


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Training Accuracy',
              'type': 'scatter',
              'uid': '97afa450-b61e-4685-97cf-633e6c8e7921',
              'x': [],
              'y': []},
             {'mode': 'lines',
              'name': 'Testing Accuracy',
              'type': 'scatter',
              'uid': 'e0f00ac8-e4b2-4317-a943-9e837c1af382',
              'x': [],
              'y': []},
             {'line': {'color': 'gray', 'dash': 'dash'},
              'mode': 'lines',
              'name': 'Test Regression',
              'type': 'scatter',
              'uid': '8ece934a-ce6d-49c1-b442-ae086eb84385',
              'x': [],
              'y': []}],
    'layout': {'annotations': [{'font': {'size': 18},
                                'showarrow': False,
                                'text': 'Train: - , Test: -',
                                'x': 0.5,
                                'xref': 'paper',
                          